In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import Counter, defaultdict
import pickle
from pprint import pprint
import random
import re
import string

import amrlib
import dill
from nltk.stem import WordNetLemmatizer
import pandas as pd
import penman

from annotate_datasets import align_tokens_to_sentence

stanza_nlp = stanza.Pipeline(lang="en")r

## Constants

In [3]:
datasets = ["risec", "japflow", "chemu", "mscorpus"]

## EDA: what is unaligned?

In [4]:
unaligned_triples = {}
unaligned_instances = {}

# check unaligned words: if they occur enough, they're probably AMR-model fillins
unaligned_words = []

node_pattern = re.compile(r"z\d+")
pbf_pattern = re.compile(r"(\w+-)+(\d){2}")

with open("/usr0/home/sgururaj/miniconda3/envs/amr/lib/python3.7/site-packages/amrlib/models/parse_spring/resources/additions.txt") as f:
    additions = f.read().split("\n")

with open("code/amr/amr_keywords.txt") as f:
    amr_keywords = [word.strip() for word in f.readlines() if word.strip() and not word.startswith("#")]

def is_unit_instance(triple, graph):
    s, r, t = triple
    is_instance = r == ":instance"
    is_unit_instance = any([parent_triple[1] in {":unit", ":scale"} and parent_triple[2] == s for parent_triple in graph.triples])
    return is_instance and is_unit_instance

def should_be_unaligned(triple, graph):
    # if this is a node like z1 --> z2, or z1 --> temperature_quantity
    is_node2node = bool(node_pattern.fullmatch(triple[0])) and bool(node_pattern.match(triple[2]))
    # if this is an AMR specifier node
    is_amr_keyword = triple[1] == ":instance" and triple[2] in additions or triple[2] in amr_keywords
    # if this is an intervening node triple
    is_name_triple = bool(node_pattern.fullmatch(triple[0])) and triple[2] == "name"
    # if this is a "you" truple
    is_imperative = triple[2] == "you" or triple[1] == ":mode" and triple[2] == "imperative"

    return is_node2node or is_amr_keyword or is_name_triple or is_unit_instance(triple, graph) or is_imperative


def is_propbank_frame(node_str):
    return bool(pbf_pattern.fullmatch(node_str))


for dataset in datasets:
    with open(f"/scratch/sgururaj/flow_graphs/{dataset}/amr_train.pkl", "rb") as f:
        amr_data = pickle.load(f)
    
    unaligned_instances[dataset] = []
    unaligned_triples[dataset] = []

    for doc in amr_data:
        for sentence in doc:
            amr_graph = sentence["graph"]
            text = sentence["text"]
            if amr_graph is None or amr_graph.triples[0][0] is None:
                continue
            alignments = penman.surface.alignments(amr_graph)
            for triple in amr_graph.triples:
                if triple not in alignments and not should_be_unaligned(triple, amr_graph):
                    unaligned_triples[dataset].append((triple, sentence))
                    if triple[1] == ":instance":
                        unaligned_instances[dataset].append((triple, sentence))
                        unaligned_words.append(triple[2])

In [5]:
word_counter = Counter(unaligned_words)
word_counter.most_common(10)

[('and', 1555),
 ('mean-01', 781),
 ('between', 589),
 ('dry-02', 491),
 ('after', 386),
 ('add-02', 294),
 ('example', 232),
 ('equal-01', 225),
 ('slash', 175),
 ('heat-01', 169)]

In [6]:
error_counts = {}

mystery = []
hyphens = []

for dataset in datasets:
    err_counts = defaultdict(lambda: 0) 
    unaligned = unaligned_instances[dataset]
    for (s,r,t), _ in unaligned:
        if t in additions or t in amr_keywords:
            err_counts["keywords"] += 1
        elif pbf_pattern.match(t):
            err_counts["pbf"] += 1
        elif "-" in t:
            err_counts["hyphenated"] += 1
            hyphens.append(t)
        elif t == "you":
            err_counts["you"] +=1
        else:
            mystery.append(t)
    error_counts[dataset] = err_counts, len(unaligned)

error_counts

{'risec': (defaultdict(<function __main__.<lambda>()>,
              {'pbf': 765, 'hyphenated': 6}),
  1139),
 'japflow': (defaultdict(<function __main__.<lambda>()>,
              {'pbf': 2280, 'hyphenated': 18}),
  3629),
 'chemu': (defaultdict(<function __main__.<lambda>()>,
              {'pbf': 2302, 'hyphenated': 275}),
  5601),
 'mscorpus': (defaultdict(<function __main__.<lambda>()>,
              {'pbf': 1068, 'hyphenated': 129}),
  2462)}

Takeaway here: some hyphenated words do exist: match against these by replacing hyphens and seeing if you can find a string match. Other hyphenated words are weird, and do not appear in the AMR spec, so not sure if they're valid to use or add there. The vast majority of unaligned words seem actually to be just that: unaligned words.

In [7]:
for dataset in datasets:
    unaligned = unaligned_instances[dataset]
    print(Counter([t for (_, _, t), _ in unaligned if not pbf_pattern.match(t) and t not in additions and t not in amr_keywords]).most_common(20))

[('and', 100), ('between', 57), ('juice', 25), ('cinnamon', 20), ('bubbly', 5), ('fryer', 4), ('cayenne', 4), ('chill', 4), ('parmesan', 4), ('sprinkle', 4), ('more', 4), ('we', 4), ('ratio-of', 3), ('oat', 3), ('aluminum', 3), ('worcestershire', 3), ('cilantro', 3), ('gradual', 3), ('layer', 3), ('cumin', 3)]
[('and', 304), ('between', 151), ('gas', 130), ('juice', 81), ('or', 75), ('meanwhile', 27), ('slash', 26), ('ice', 24), ('lid', 21), ('cumin', 18), ('thyme', 18), ('cinnamon', 16), ('zest', 15), ('wok', 15), ('after', 13), ('more', 13), ('cheese', 12), ('then', 11), ('pan', 10), ('i', 10)]
[('and', 944), ('between', 305), ('after', 277), ('example', 232), ('compound', 121), ('slash', 110), ('step', 105), ('ratio-of', 72), ('water', 62), ('or', 61), ('then', 59), ('this', 48), ('intermediate', 38), ('tetrahydrofuran', 36), ('hexane', 35), ('small-methanol', 32), ('method', 29), ('angle-quantity', 29), ('string-entity', 28), ('column', 25)]
[('and', 207), ('after', 94), ('between'

In [8]:
wnl = WordNetLemmatizer()

In [9]:
wnl.lemmatize("leaves")

'leaf'

In [10]:
## visual inspection cell

instance = random.choice(unaligned_instances["mscorpus"])
sentence = instance[1]["text"]
graph = instance[1]["graph"]

print(instance[1]["text"])
print(instance[0])
print(penman.encode(instance[1]["graph"]))

It is worth to mention that POFA is a not toxic waste material in terms of heavy metals leachability [51].
('z8', ':instance', 'possible-01')
(z1 / worth-02~e.2
    :arg2 (z2 / mention-01~e.4
              :arg1 (z3 / material~e.12
                        :polarity~e.9 -~e.9
                        :mod (z4 / waste~e.11)
                        :mod (z5 / toxic~e.10)
                        :domain (z6 / small-molecule
                                    :name (z7 / name
                                              :op1 "pofa"))
                        :topic (z8 / possible-01
                                   :arg1 (z9 / leach-01~e.18
                                             :arg1 (z10 / metal~e.17
                                                        :mod (z11 / heavy~e.16))))
                        :arg1-of (z12 / describe-01
                                      :arg0 (z13 / publication
                                                 :arg1-of (z14 / cite-01
              

In [11]:
penman.surface.alignments(graph)

{('z1', ':instance', 'worth-02'): Alignment((2,), prefix='e.'),
 ('z2', ':instance', 'mention-01'): Alignment((4,), prefix='e.'),
 ('z3', ':instance', 'material'): Alignment((12,), prefix='e.'),
 ('z3', ':polarity', '-'): Alignment((9,), prefix='e.'),
 ('z4', ':instance', 'waste'): Alignment((11,), prefix='e.'),
 ('z5', ':instance', 'toxic'): Alignment((10,), prefix='e.'),
 ('z9', ':instance', 'leach-01'): Alignment((18,), prefix='e.'),
 ('z10', ':instance', 'metal'): Alignment((17,), prefix='e.'),
 ('z11', ':instance', 'heavy'): Alignment((16,), prefix='e.')}

In [12]:
def get_unaligned_tokens(aligned_tokens, graph):
    is_aligned = [False] * len(aligned_tokens)
    for alignment in penman.surface.alignments(graph).values():
        for index in alignment.indices:
            is_aligned[index] = True
    return [token for token_aligned, token in zip(is_aligned, aligned_tokens) if not token_aligned]

In [13]:
def search_for_aligning_token(sentence, node, graph, lemmatizer):

    default_pos = "n"
    if pbf_pattern.match(node):
        segments = node.split("-")
        if len(segments) > 2:
            return None
        else: 
            nodes = [segments[0]]
            word_lemmas = [lemmatizer.lemmatize(node.replace('"', ""), pos="v") for node in nodes]
            default_pos = "v"
    elif "-" in node:
        nodes = node.split("-")
        word_lemmas = [lemmatizer.lemmatize(node.strip('"')) for node in nodes]
    else:
        nodes = [node]
        word_lemmas = [lemmatizer.lemmatize(node.strip('"')) for node in nodes]

    char_aligned_tokens = align_tokens_to_sentence([token for token in re.split("\s+", sentence) if token.strip()], sentence)
    unaligned_tokens = get_unaligned_tokens(char_aligned_tokens, graph)
    newly_aligned_tokens = []

    for word_lemma in word_lemmas:
        for unaligned_token in unaligned_tokens:
            unaligned_text = unaligned_token.token_str
            
            unaligned_lemma = lemmatizer.lemmatize(unaligned_text.lower().strip(string.punctuation), pos=default_pos)
            if unaligned_lemma == word_lemma:
                # print(word_lemma, unaligned_lemma)
                newly_aligned_tokens.append(unaligned_token)
    if not newly_aligned_tokens:
        return None
    else:
        return newly_aligned_tokens

In [14]:
sentence = instance[1]["text"]
graph = instance[1]["graph"]

print(instance[1]["text"])
print(instance[0])
print(search_for_aligning_token(sentence, instance[0][2], graph, wnl))
print()

print(penman.encode(instance[1]["graph"]))

It is worth to mention that POFA is a not toxic waste material in terms of heavy metals leachability [51].
('z8', ':instance', 'possible-01')
None

(z1 / worth-02~e.2
    :arg2 (z2 / mention-01~e.4
              :arg1 (z3 / material~e.12
                        :polarity~e.9 -~e.9
                        :mod (z4 / waste~e.11)
                        :mod (z5 / toxic~e.10)
                        :domain (z6 / small-molecule
                                    :name (z7 / name
                                              :op1 "pofa"))
                        :topic (z8 / possible-01
                                   :arg1 (z9 / leach-01~e.18
                                             :arg1 (z10 / metal~e.17
                                                        :mod (z11 / heavy~e.16))))
                        :arg1-of (z12 / describe-01
                                      :arg0 (z13 / publication
                                                 :arg1-of (z14 / cite-01
        

In [15]:
alignment_counts = {}



for dataset in datasets:
    with open(f"/scratch/sgururaj/flow_graphs/{dataset}/amr_train.pkl", "rb") as f:
        amr_data = pickle.load(f)
    alignment_counts[dataset] = defaultdict(lambda: 0)

    for doc in amr_data:
        for sentence in doc:
            amr_graph = sentence["graph"]
            text = sentence["text"]
            if amr_graph is None or amr_graph.triples[0][0] is None:
                continue
            alignments = penman.surface.alignments(amr_graph)
            for triple in amr_graph.triples:
                alignment_counts[dataset]["triples"] += 1
                if triple not in alignments and should_be_unaligned(triple, amr_graph):
                    alignment_counts[dataset]["unaligned_fine"] += 1
                if triple not in alignments and not should_be_unaligned(triple, amr_graph):
                    alignment_counts[dataset]["unaligned_concerning"] += 1

                    new_alignment = search_for_aligning_token(text, triple[2], amr_graph, wnl)
                    if new_alignment is None:
                        alignment_counts[dataset]["concerning_unfixed"] += 1
                    else:
                        alignment_counts[dataset]["concerning_fixed"] += 1

pd.DataFrame(alignment_counts)

,risec,japflow,chemu,mscorpus
triples,15750,54239,129382,47350
unaligned_fine,7826,31171,79960,28751
unaligned_concerning,2039,3936,17332,6131
concerning_fixed,1561,2599,8652,3102
concerning_unfixed,478,1337,8680,3029
